# score : 0.98996

# Lib

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models

from PIL import Image, ImageOps, ImageEnhance
import numbers

import time
import copy

# Device

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Data

In [4]:
# colab ver
test = pd.read_csv('/content/drive/MyDrive/digit/Data/test.csv')
train = pd.read_csv('/content/drive/MyDrive/digit/Data/train.csv')
sample = pd.read_csv('/content/drive/MyDrive/digit/Data/sample_submission.csv')

# Local ver
# test = pd.read_csv('./Data/test.csv')
# train = pd.read_csv('./Data/train.csv')
# sample = pd.read_csv('./Data/sample_submission.csv')

print('test.csv')
print(f'test shape : {test.shape}')
print(f'first column : {test.columns[0]}, last column : {test.columns[-1]}')
print()
print('train.csv')
print(f'train shape : {train.shape}')
print(f'first column : {train.columns[0]}, last column : {train.columns[-1]}')
print()
print('sample_submission.csv')
print(f'sample shape : {sample.shape}')
print(f'first column : {sample.columns[0]}, last column : {sample.columns[-1]}')
print()

test.csv
test shape : (28000, 784)
first column : pixel0, last column : pixel783

train.csv
train shape : (42000, 785)
first column : label, last column : pixel783

sample_submission.csv
sample shape : (28000, 2)
first column : ImageId, last column : Label



## Data structure

In [5]:
class CustomDataset(Dataset):
    # check 1 : train, 0 : test
    def __init__(self, data_path, check):
        df = pd.read_csv(data_path)
        
        if check:
            self.transform = transforms.Compose([
                transforms.ToPILImage(), 
                transforms.RandomRotation(degrees=20),
                transforms.ToTensor(), 
                transforms.Normalize(mean=(0.5,), std=(0.5,))
            ])
        else:
            self.transform = transforms.Compose([
                transforms.ToPILImage(), 
                transforms.ToTensor(), 
                transforms.Normalize(mean=(0.5,), std=(0.5,))
            ])
        
        # test data
        if len(df.columns) == 784:
            # reshape(), (28000, 784) -> (28000, 28, 28)
            # -1은 이전 값
            self.X = df.values.reshape((-1, 28, 28)).astype(np.uint8)[:,:,:,None]
            self.y = None
        # train data    
        else:
            self.X = df.iloc[:, 1:].values.reshape((-1, 28, 28)).astype(np.uint8)[:,:,:,None]
            self.y = torch.from_numpy(df.iloc[:, 0].values)
        
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        if self.y is not None: 
            return self.transform(self.X[idx]), self.y[idx]
        else:
            return self.transform(self.X[idx])

# Data loader

In [6]:
# colab ver
train_path = '/content/drive/MyDrive/digit/Data/train.csv'
test_path = '/content/drive/MyDrive/digit/Data/test.csv'
# Local ver
# train_path = './Data/train.csv'
# test_path = './Data/test.csv'

train_dataset = CustomDataset(train_path, 1)
test_dataset = CustomDataset(test_path, 0)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=64,
                                           shuffle=False)

# Network

In [14]:
model_ft = models.resnet50(pretrained=True)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [15]:
# channel setting -> gray and color image
model_ft.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

num_ftrs = model_ft.fc.in_features
# Linear(in_ftrs, out_ftrs)
# out_ftrs가 0~9 이기에 10
model_ft.fc = torch.nn.Linear(num_ftrs, 10, bias=True)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model_ft.parameters(), lr=0.003)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Train

In [18]:
def train(model, criterion, optimizer, scheduler, num_epoches = 10):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    
    for epoch in range(num_epoches):
        print(f'Epoch: {epoch} / {num_epoches-1}')
        print('-'*10)
        
        model.train()
        
        train_loss = 0.0
        train_acc = 0.0
        train_total = 0
        
        
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            _, preds = torch.max(outputs.data, 1)
            
            
            train_loss += loss.item() * inputs.size(0)
            train_acc += (preds == labels).sum().item()
            train_total += labels.size(0)
            
        scheduler.step()
            
        epoch_loss = train_loss / train_total
        epoch_acc = train_acc / train_total
        
        print(f'epoch loss : {epoch_loss:.4f}, epoch acc : {epoch_acc:.4f}')
        
        # better weight save
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
    
    # time
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    
    # best weight of model
    model.load_state_dict(best_model_wts)
    return model
    
            

In [19]:
model_ft = train(model_ft, criterion, optimizer, exp_lr_scheduler, num_epoches = 10)

Epoch: 0 / 9
----------
epoch loss : 0.3075, epoch acc : 0.9046
Epoch: 1 / 9
----------
epoch loss : 0.3586, epoch acc : 0.8979
Epoch: 2 / 9
----------
epoch loss : 0.1490, epoch acc : 0.9550
Epoch: 3 / 9
----------
epoch loss : 0.1066, epoch acc : 0.9685
Epoch: 4 / 9
----------
epoch loss : 0.1434, epoch acc : 0.9610
Epoch: 5 / 9
----------
epoch loss : 0.0877, epoch acc : 0.9746
Epoch: 6 / 9
----------
epoch loss : 0.1293, epoch acc : 0.9647
Epoch: 7 / 9
----------
epoch loss : 0.0537, epoch acc : 0.9831
Epoch: 8 / 9
----------
epoch loss : 0.0441, epoch acc : 0.9865
Epoch: 9 / 9
----------
epoch loss : 0.0398, epoch acc : 0.9870
Training complete in 7m 4s


# Predict

In [20]:
model_ft.eval()
test_preds = []

with torch.no_grad():
    for inputs in test_loader:
        inputs = inputs.to(device)
        
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs.data, 1)
        test_preds.extend(preds.cpu().numpy())

In [21]:
sample['Label'] = test_preds
sample.to_csv('final.csv', index=False)